In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import MultipleLocator
import time
import datetime
from datetime import timedelta
import scipy.stats as stats
from scipy.stats import pointbiserialr

In [2]:
ID_57_2021=pd.read_csv('C:\\2_Studium\\SS2022\\ADP\\AZO\\AZO\\2021\\TagID_057_2021.csv',dtype= {'CreateDateTime': str, 'TagValue': np.float64, 'TagId': np.int32},parse_dates=['CreateDateTime'],index_col='Unnamed: 0')
ID_159_2021=pd.read_csv('C:\\2_Studium\\SS2022\\ADP\\AZO\\AZO\\2021\\TagID_159_2021.csv',dtype= {'CreateDateTime': str, 'TagValue': np.float64, 'TagId': np.int32},parse_dates=['CreateDateTime'],index_col='Unnamed: 0')
ID_158_2021=pd.read_csv('C:\\2_Studium\\SS2022\\ADP\\AZO\\AZO\\2021\\TagID_158_2021.csv',dtype= {'CreateDateTime': str, 'TagValue': np.float64, 'TagId': np.int32},parse_dates=['CreateDateTime'],index_col='Unnamed: 0')

C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
Daten_1=ID_57_2021
Daten_2=ID_159_2021

# Daten_1=Daten_1[(Daten_1['CreateDateTime']>'2021-03-24 16:06:00')&(Daten_1['CreateDateTime']<'2021-03-25 00:06:00')]
# Daten_2=Daten_2[(Daten_2['CreateDateTime']>'2021-03-24 16:06:00')&(Daten_2['CreateDateTime']<'2021-03-25 00:06:00')]
DatenJion=Daten_1.set_index('CreateDateTime').join(Daten_2.set_index('CreateDateTime'),lsuffix='_l', rsuffix='_r',how='outer')
DatenJion['TagValue_l']=DatenJion['TagValue_l'].interpolate(method='nearest')
DatenJion=DatenJion.fillna(method="ffill")
DatenJion=DatenJion.dropna()

DatenJion['shift_1']=DatenJion.TagValue_l.shift(periods=1)
DatenJion['shift_-1']=DatenJion.TagValue_l.shift(periods=-1)
DatenJion['shift_2']=DatenJion.TagValue_l.shift(periods=2)
DatenJion['shift_-2']=DatenJion.TagValue_l.shift(periods=-2)
DatenJion=DatenJion.reset_index('CreateDateTime')
DatenJion

,CreateDateTime,TagValue_l,TagId_l,TagValue_r,TagId_r,shift_1,shift_-1,shift_2,shift_-2
0,2021-02-17 14:05:12.214,-1.533565,57.0,0.0,159.0,NaN,-1.533565,NaN,-1.475694
1,2021-02-17 14:05:12.217,-1.533565,57.0,0.0,159.0,-1.533565,-1.475694,NaN,-1.533565
2,2021-02-17 14:05:13.870,-1.475694,57.0,0.0,159.0,-1.533565,-1.533565,-1.533565,-1.475694
3,2021-02-17 14:05:14.088,-1.533565,57.0,0.0,159.0,-1.475694,-1.475694,-1.533565,-1.533565
4,2021-02-17 14:05:14.286,-1.475694,57.0,0.0,159.0,-1.533565,-1.533565,-1.475694,-1.475694
...,...,...,...,...,...,...,...,...,...
21275491,2021-10-07 08:35:50.674,-1.707176,57.0,0.0,159.0,-1.678241,-1.736111,-1.707176,-1.678241
21275492,2021-10-07 08:35:51.258,-1.736111,57.0,0.0,159.0,-1.707176,-1.678241,-1.678241,-1.736111
21275493,2021-10-07 08:35:52.535,-1.678241,57.0,0.0,159.0,-1.736111,-1.736111,-1.707176,-1.707176
21275494,2021-10-07 08:35:53.572,-1.736111,57.0,0.0,159.0,-1.678241,-1.707176,-1.736111,NaN


In [4]:
print(datetime.datetime.now())
df=DatenJion
for indexs in df.index:
        if df.loc[indexs, "TagValue_r"]==1:
            df.loc[indexs, "TagValue_l"]=min(df.loc[indexs, "TagValue_l"],df.loc[indexs, "shift_1"],df.loc[indexs, "shift_-1"],df.loc[indexs, "shift_2"],df.loc[indexs, "shift_-2"])
            df.loc[indexs-1, "TagValue_r"]=np.nan
            df.loc[indexs+1, "TagValue_r"]=np.nan
            df.loc[indexs-2, "TagValue_r"]=np.nan
            df.loc[indexs+2, "TagValue_r"]=np.nan
print(datetime.datetime.now())

2022-09-02 15:23:53.375341
2022-09-02 15:29:37.789662


In [5]:
df

,CreateDateTime,TagValue_l,TagId_l,TagValue_r,TagId_r,shift_1,shift_-1,shift_2,shift_-2
0,2021-02-17 14:05:12.214,-1.533565,57.0,0.0,159.0,NaN,-1.533565,NaN,-1.475694
1,2021-02-17 14:05:12.217,-1.533565,57.0,0.0,159.0,-1.533565,-1.475694,NaN,-1.533565
2,2021-02-17 14:05:13.870,-1.475694,57.0,0.0,159.0,-1.533565,-1.533565,-1.533565,-1.475694
3,2021-02-17 14:05:14.088,-1.533565,57.0,0.0,159.0,-1.475694,-1.475694,-1.533565,-1.533565
4,2021-02-17 14:05:14.286,-1.475694,57.0,0.0,159.0,-1.533565,-1.533565,-1.475694,-1.475694
...,...,...,...,...,...,...,...,...,...
21275491,2021-10-07 08:35:50.674,-1.707176,57.0,0.0,159.0,-1.678241,-1.736111,-1.707176,-1.678241
21275492,2021-10-07 08:35:51.258,-1.736111,57.0,0.0,159.0,-1.707176,-1.678241,-1.678241,-1.736111
21275493,2021-10-07 08:35:52.535,-1.678241,57.0,0.0,159.0,-1.736111,-1.736111,-1.707176,-1.707176
21275494,2021-10-07 08:35:53.572,-1.736111,57.0,0.0,159.0,-1.678241,-1.707176,-1.736111,NaN


In [6]:
DF=df[['CreateDateTime','TagValue_l','TagValue_r']]
DF=DF.dropna()
DF
pointbiserialr(DF['TagValue_r'],DF['TagValue_l'])

PointbiserialrResult(correlation=-0.08599402364197234, pvalue=0.0)

In [7]:
Daten_2=ID_158_2021
# Daten_2=Daten_2[(Daten_2['CreateDateTime']>'2021-03-24 16:06:00')&(Daten_2['CreateDateTime']<'2021-03-25 00:06:00')]
DatenJion_2=DF.set_index('CreateDateTime').join(Daten_2.set_index('CreateDateTime'),lsuffix='_l', rsuffix='_r',how='outer')
DatenJion_2=DatenJion_2.fillna(method="ffill")

DatenJion_2

,TagValue_l,TagValue_r,TagValue,TagId
CreateDateTime,,,,
2021-02-17 14:05:12.212,NaN,NaN,0.0,158.0
2021-02-17 14:05:12.214,-1.533565,0.0,0.0,158.0
2021-02-17 14:05:12.217,-1.533565,0.0,0.0,158.0
2021-02-17 14:05:13.870,-1.475694,0.0,0.0,158.0
2021-02-17 14:05:14.088,-1.533565,0.0,0.0,158.0
...,...,...,...,...
2021-10-07 08:35:50.674,-1.707176,0.0,0.0,158.0
2021-10-07 08:35:51.258,-1.736111,0.0,0.0,158.0
2021-10-07 08:35:52.535,-1.678241,0.0,0.0,158.0


In [8]:
df_clear = DatenJion_2.drop(DatenJion_2[DatenJion_2['TagValue']==0].index)
df_clear

,TagValue_l,TagValue_r,TagValue,TagId
CreateDateTime,,,,
2021-03-02 05:44:58.853,-1.909722,0.0,1.0,158.0
2021-03-02 05:44:58.854,-11.082180,1.0,1.0,158.0
2021-03-02 05:44:59.271,-1.331019,0.0,1.0,158.0
2021-03-02 05:44:59.465,-1.186343,0.0,1.0,158.0
2021-03-02 05:44:59.673,-1.273148,0.0,1.0,158.0
...,...,...,...,...
2021-08-11 16:17:03.864,-0.549769,0.0,1.0,158.0
2021-08-11 16:17:04.070,-0.434028,0.0,1.0,158.0
2021-08-11 16:17:04.278,-0.376157,0.0,1.0,158.0


In [9]:
pointbiserialr(df_clear['TagValue_r'],df_clear['TagValue_l'])

PointbiserialrResult(correlation=-0.8620825824219835, pvalue=0.0)